# Exercice :


### Par : Abdelhadi DANBA


J'ai essayé de modélisé le problème en utilisant une approche graphe.

Ensuite nous allons facilement appliquer les algorithmes de parcours d'arbre afin de trouver l'optimum.

L'optimum ici correspond à la combinaison d'offre maximisant la pertinence.


**NB:** Dans un premier temps nous avons utilisé une approche itérative (draft), une fois la difficulté du problème assimulée, nous avons passé à un niveau de modélisation avancé et abstrait (en utilisant des class), pour la restitution finale.






    
![Graphe Offre : ](https://raw.githubusercontent.com/adanba/Images/master/graphe_offre.png)

## Objet Offre : 

L'object **Offre** correspond à l'offre parcouru par le prospect.
Cette dernière est caractérisée par :

* Un nom 
* Une valeur de pertinence

In [62]:
class Offre(object):
    def __init__(self, name,value):
        """ Assumes name is a string
            Assumes value is a int
        """
        self.name = name
        self.value = value
    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def __str__(self):
        return self.name


## Objet Edge : 


Cette classe permet de modéliser s'il existe, le passage d'une offre à une autre, ce qu'on appelle arête en théorie des graphe.

In [63]:

class Edge(object):
    def __init__(self, src, dest):
        """Assumes src and dest are nodes (class Offre)"""
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return self.src.getName() + '->' + self.dest.getName()
               

## Objet Parcours : 

La classe **Parcours** est un graphe dont les noeuds coresspandent aux **Offres**

**Parcours** est utilisée pour modéliser tous les parcours possibles du prospect.







In [64]:
class Parcours(object):
    """ edges is a dict mapping each node to a list of its children"""
    
    def __init__(self):
        """Constructeur de la classe"""
        self.edges = {}
        
    def addOffre(self, offre):
        """Rajouter une offre (un noeud) au graphe"""
        if offre in self.edges:
            raise ValueError('Duplicate offre')
        else:
            self.edges[offre] = []

    def getOffre(self, name):
        """Récupérer une offre (un noeud) à partir de son nom au graphe"""
        for offre in self.edges:
            if offre.getName() == name:
                return offre
        raise NameError(name)

    def addEdge(self, edge):
        """Rajouter un chemin orienté entre deux offres (noeuds) du graphe"""
        src = edge.getSource()
        dest = edge.getDestination()
        if not (src in self.edges and dest in self.edges):
            raise ValueError('Offre not in graph')
        self.edges[src].append(dest)

    def childrenOf(self, offre):
        """Recupérer les offres filles (suivantes) d'une offre (noeud) au graphe"""
        return self.edges[offre]
    
    def hasOffre(self, offre):
        """Voir si une offre (noeud) existe dans le graphe"""
        return offre in self.edges
    
    def __str__(self):
        """Permet de faire des output text jolie de notre graphe """
        result = ''
        for src in self.edges:
            for dest in self.edges[src]:
                result = result + src.getName() + '->'\
                         + dest.getName() + '\n'
        return result[:-1]


## Construction du graphe :

La fonction **buildOffreParcour** permet de créer le graphe correspandant, à partir d'une liste d'offres successives.

Cette fonction intégrer la possibilité de changer le saut, c-à-d le nombre d'offres possible à ignorer.

Les arrêtes du graphe d'offres sont déterministes en fonction de la règle de saut adoptée.


In [66]:
def buildOffreParcour(offres, n_rollement=4):
    """Construit le graphe d'offres à partir d'une liste d'Offres"""
    g = Parcours()
    for name in range(len(offres)): 
        g.addOffre(Offre(str(name), offres[name]))
    for start_offre in range(len(offres)):
        #print "====> Node : ", start_offre
        for next_offre in range(start_offre + 1, start_offre + 2 + n_rollement):
            if next_offre + 1 > len(offres):
                #print "Break Next Node : ", next_offre
                break
            #print "========> Next Node : ", next_offre
            g.addEdge(Edge(g.getOffre(str(start_offre)), g.getOffre(str(next_offre))))
    return g


## Fonctions utilitaires

* La fonction **Pertinence** :  calcule la Pertinence **d'un parcours prospect**, c-à-d un chemin d'offres  (liste d'offres)


* La fonction **printPath** : permet de faire des outputs texte d'un parcours. 

In [67]:
def Pertinence(Parcours, chemin):
    """Calcul la pertinence d'un chemin (parcours prospect)"""
    result = 0
    for name_offre in chemin:
        result += Parcours.getOffre(name_offre).getValue()
    return result 


def printPathOffres(Parcours, chemin):
    """On suppose que chemin est une liste d'offres"""
    result = ''
    for i in range(len(chemin)):
        result = result + Parcours.getOffre(chemin[i]).getName()
        if i != len(chemin) - 1:
            result = result + '->'
    return result

## Algorithme DFS (Depth First Search) :


Nous avons utilisé l'algorithme de parcours en profondeur (DFS), pour trouver le chemin qui maximise la pertinence en partant de la première offre jusqu'à la dernière.



In [68]:
def DFS(parcours, start_offre, end_offre, chemin, cheminPertinent, toPrint = False):
    """ On suppose parcours une instance de Parcours graph;
        chemin and cheminPertinent sont des listes d'offres
        start_offre et end_offre sont des Offres (nodes);
        retourne cheminPertinent qui maximise la pertinence 
        et ce en allant de start_offre à end_offre"""
    
    chemin = chemin + [start_offre]
    
    if toPrint:
        print('Current DFS path:', printPathOffres(parcours, chemin), "    Value : ", Pertinence(parcours, chemin))
    
    if start_offre == end_offre:
        return chemin
    
    for offre in parcours.childrenOf(parcours.getOffre(start_offre)):
        newChemin = DFS(parcours, offre.getName(), end_offre, chemin, cheminPertinent, toPrint)
        if newChemin != None:
            if cheminPertinent == None or Pertinence(parcours, newChemin) > Pertinence(parcours, cheminPertinent):
                cheminPertinent = newChemin 

    return cheminPertinent


def pertinenceMaximale(parcours, start_offre, end_offre, toPrint = False):
    """ On suppose que parcours une instance de Parcours graph; 
        start_offre et end_offre sont des Offres (nodes);
        retourne cheminPertinent qui maximise la pertinence 
        et ce en allant de start_offre à end_offre"""
    return DFS(parcours, start_offre, end_offre, [], None, toPrint)


# Exemple 1 :


Exemple sur une liste de 6 offres  (fourni dans ** offres.txt**)

In [84]:
listOffres = [1,0,-1,3,-1,-2]

parcours = buildOffreParcour(listOffres)

In [85]:
cheminPertinent = pertinenceMaximale(parcours, '0', '5', toPrint = True)

('Current DFS path:', '0', '    Value : ', 1)
('Current DFS path:', '0->1', '    Value : ', 1)
('Current DFS path:', '0->1->2', '    Value : ', 0)
('Current DFS path:', '0->1->2->3', '    Value : ', 3)
('Current DFS path:', '0->1->2->3->4', '    Value : ', 2)
('Current DFS path:', '0->1->2->3->4->5', '    Value : ', 0)
('Current DFS path:', '0->1->2->3->5', '    Value : ', 1)
('Current DFS path:', '0->1->2->4', '    Value : ', -1)
('Current DFS path:', '0->1->2->4->5', '    Value : ', -3)
('Current DFS path:', '0->1->2->5', '    Value : ', -2)
('Current DFS path:', '0->1->3', '    Value : ', 4)
('Current DFS path:', '0->1->3->4', '    Value : ', 3)
('Current DFS path:', '0->1->3->4->5', '    Value : ', 1)
('Current DFS path:', '0->1->3->5', '    Value : ', 2)
('Current DFS path:', '0->1->4', '    Value : ', 0)
('Current DFS path:', '0->1->4->5', '    Value : ', -2)
('Current DFS path:', '0->1->5', '    Value : ', -1)
('Current DFS path:', '0->2', '    Value : ', 0)
('Current DFS path:'

In [91]:
print "Le resultat donnée par l'algorithme : ", printPathOffres(parcours, cheminPertinent)

print "La valeur du chemin le plus pertinent (Max) : ", Pertinence(parcours, cheminPertinent)

Le resultat donnée par l'algorithme :  0->1->3->5->6->8
La valeur du chemin le plus pertinent (Max) :  2


# Exemple 2:


Un autre exemple sur une liste de 8 offres 

In [87]:
listOffres = [1,0,-1,3,-2,0,0,-1,-2]

parcours = buildOffreParcour(listOffres)

In [88]:
cheminPertinent = pertinenceMaximale(parcours, '0', '8', toPrint = True)

('Current DFS path:', '0', '    Value : ', 1)
('Current DFS path:', '0->1', '    Value : ', 1)
('Current DFS path:', '0->1->2', '    Value : ', 0)
('Current DFS path:', '0->1->2->3', '    Value : ', 3)
('Current DFS path:', '0->1->2->3->4', '    Value : ', 1)
('Current DFS path:', '0->1->2->3->4->5', '    Value : ', 1)
('Current DFS path:', '0->1->2->3->4->5->6', '    Value : ', 1)
('Current DFS path:', '0->1->2->3->4->5->6->7', '    Value : ', 0)
('Current DFS path:', '0->1->2->3->4->5->6->7->8', '    Value : ', -2)
('Current DFS path:', '0->1->2->3->4->5->6->8', '    Value : ', -1)
('Current DFS path:', '0->1->2->3->4->5->7', '    Value : ', 0)
('Current DFS path:', '0->1->2->3->4->5->7->8', '    Value : ', -2)
('Current DFS path:', '0->1->2->3->4->5->8', '    Value : ', -1)
('Current DFS path:', '0->1->2->3->4->6', '    Value : ', 1)
('Current DFS path:', '0->1->2->3->4->6->7', '    Value : ', 0)
('Current DFS path:', '0->1->2->3->4->6->7->8', '    Value : ', -2)
('Current DFS path:

In [90]:
print "Le resultat donnée par l'algorithme : ", printPathOffres(parcours, cheminPertinent)

print "La valeur du chemin le plus pertinent (Max) : ", Pertinence(parcours, cheminPertinent)

Le resultat donnée par l'algorithme :  0->1->3->5->6->8
La valeur du chemin le plus pertinent (Max) :  2


# Conclusion et limitations : 

* La méthode implémentée ici est d'une compléxité O(V+E) (où : V est le nombre d'offres et E nombre de parcours possible)

* Lors de la comparaison de deux chemins ces derniers peuvent avoir la même pertinence, dans ce cas d'égalité on peut maintenir le chemin le plus court si nous sommes dans un contexte d'affichage publicitaire, etc. Dans ma solution je me suis contenté juste de prendre la premier.

